# SVM Simulation

## Data Preparation

In [1]:
# add required libraries
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [7]:
!pip install -q nltk
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# set random seed to reproduce the same result every time
np.random.seed(500)

In [14]:
# import the corpus
Corpus = pd.read_csv("500_Reddit_users_posts_labels.csv", encoding='latin-1')

## Data Preprocessing

### Tokenization & Word Stemming/Lemmatization

In [16]:
# Step - a : Remove blank rows if any.
Corpus['Post'].dropna(inplace=True)

# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['Post'] = [entry.lower() for entry in Corpus['Post']]

# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['Post']= [word_tokenize(entry) for entry in Corpus['Post']]

# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['Post']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

In [17]:
!pip install -q tabulate

import pandas as pd
from tabulate import tabulate

# display a few rows of the preprocessed text
print(tabulate(Corpus.head(), headers='keys', tablefmt='psql'))

+----+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Split dataset

In [37]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(Corpus['text_final'],Corpus['Label'],test_size=0.3, random_state=42)

### Encoding

In [38]:
Encoder = LabelEncoder()
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.fit_transform(y_test)

### Word Vectorization

In [40]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(X_test)

In [41]:
# see the vocabulary that it has learned from the corpus
print(Tfidf_vect.vocabulary_)

{'viable': 4762, 'option': 3045, 'youll': 4986, 'leave': 2507, 'wife': 4898, 'behind': 423, 'youd': 4985, 'pain': 3095, 'beyond': 452, 'suck': 4259, 'worry': 4946, 'money': 2830, 'know': 2455, 'first': 1695, 'hand': 1973, 'definitely': 1126, 'feel': 1658, 'hopeless': 2107, 'seem': 3829, 'tire': 4499, 'aware': 339, 'might': 2771, 'need': 2909, 'chip': 728, 'financially': 1687, 'time': 4495, 'issue': 2378, 'even': 1510, 'hour': 2127, 'asthenia': 292, 'could': 953, 'alleviate': 146, 'lot': 2607, 'pressure': 3342, 'meantime': 2712, 'get': 1861, 'shit': 3912, 'together': 4506, 'write': 4962, 'resume': 3667, 'tomorrow': 4512, 'excuse': 1550, 'do': 1280, 'send': 3841, 'whether': 4881, 'believe': 427, 'sort': 4080, 'powerful': 3310, 'force': 1744, 'govern': 1895, 'thing': 4462, 'really': 3532, 'work': 4938, 'big': 456, 'test': 4427, 'pull': 3450, 'try': 4593, 'stay': 4174, 'positive': 3289, 'everything': 1521, 'hard': 1991, 'appreciate': 243, 'notion': 2966, 'meet': 2731, 'someone': 4064, 'els

## Multi-class Classification (One vs Rest)

In [21]:
!pip install -q scikit-learn

In [24]:
# import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import ConfusionMatrixDisplay
from mlxtend.plotting import plot_decision_regions

In [32]:
# save the preprocessed text data into a new file for reusability
Corpus.to_csv('500_Reddit_users_posts_labels_preprocessed.csv', index=False)

In [33]:
# load the preprocessed text data
Corpus = pd.read_csv("500_Reddit_users_posts_labels_preprocessed.csv", encoding='latin-1')

In [34]:
# create an instance of the SVM model
svm = LinearSVC(random_state=42)

In [35]:
# make SVM an OvR classifier
ovr_classifier = OneVsRestClassifier(svm)

In [36]:
# fit the data to the OvR classifier
ovr_classifier = ovr_classifier.fit(X_train, y_train)

ValueError: could not convert string to float: "['dont', 'know', 'dbt', 'would', 'mind', 'explain', 'little', 'bit', 'really', 'sorry', 'hear', 'someone', 'suppose', 'help', 'actually', 'really', 'mean', 'thats', 'definitely', 'right', 'fair', 'people', 'person', 'talk', 'im', 'honestly', 'sure', 'shun', 'sound', 'like', 'dont', 'know', 'either', 'im', 'sure', 'would', 'anything', 'plan', 'sound', 'really', 'pain', 'make', 'sure', 'understand', 'ish', 'cut', 'arm', 'total', 'im', 'tire', 'concerned', 'safety', 'symptom', 'wake', 'life', 'bad', 'dream', 'sound', 'like', 'tire', 'scared', 'hit', 'rock', 'bottom', 'also', 'pleasant', 'thing', 'experience', 'dbt', 'lady', 'one', 'conduct', 'therapy', 'session', 'sound', 'like', 'feel', 'really', 'scar', 'much', 'want', 'kill', 'im', 'concern', 'safety', 'especially', 'since', 'say', 'youve', 'carry', 'plan', 'suicide', 'chance', 'could', 'talk', 'bet', 'would', 'make', 'really', 'tire', 'day', 'row', 'lot', 'day', 'work', 'hear', 'youre', 